### Домашнее задание. Классификация: Логистическая регрессия и SVM.


Имеются данные adult.csv (см. в материалах к занятию).


Целевой переменной является уровень дохода income (крайний правый столбец).


Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html

Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.
Готовый ноутбук выложить на гитхаб и прислать ссылку.

In [96]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [9]:
# читаем датасет
df = pd.read_csv('adult.csv')
df.head(10)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
6,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


In [12]:
# смотрим на значения искомой переменной
df.income.unique()

array(['<=50K', '>50K'], dtype=object)

In [20]:
# здесь я забыл по LabelEncoder, но потом я его применил
df['num_income'] = df.income.transform(lambda x: 0 if x == '<=50K' else 1)
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,num_income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,0
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,1
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,1
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K,0


In [22]:
# изучаем структуру датасета
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 16 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     48842 non-null object
income             48842 non-null object
num_income         48842 non-null int64
dtypes: int64(7), object(9)
memory usage: 6.0+ MB


In [26]:
# изучаем структуру датасета, смотрим уникальные значения к колонках
for col in df:
    print(df[col].unique())

[25 38 28 44 18 34 29 63 24 55 65 36 26 58 48 43 20 37 40 72 45 22 23 54
 32 46 56 17 39 52 21 42 33 30 47 41 19 69 50 31 59 49 51 27 57 61 64 79
 73 53 77 80 62 35 68 66 75 60 67 71 70 90 81 74 78 82 83 85 76 84 89 88
 87 86]
['Private' 'Local-gov' '?' 'Self-emp-not-inc' 'Federal-gov' 'State-gov'
 'Self-emp-inc' 'Without-pay' 'Never-worked']
[226802  89814 336951 ... 129912 255835 257302]
['11th' 'HS-grad' 'Assoc-acdm' 'Some-college' '10th' 'Prof-school'
 '7th-8th' 'Bachelors' 'Masters' 'Doctorate' '5th-6th' 'Assoc-voc' '9th'
 '12th' '1st-4th' 'Preschool']
[ 7  9 12 10  6 15  4 13 14 16  3 11  5  8  2  1]
['Never-married' 'Married-civ-spouse' 'Widowed' 'Divorced' 'Separated'
 'Married-spouse-absent' 'Married-AF-spouse']
['Machine-op-inspct' 'Farming-fishing' 'Protective-serv' '?'
 'Other-service' 'Prof-specialty' 'Craft-repair' 'Adm-clerical'
 'Exec-managerial' 'Tech-support' 'Sales' 'Priv-house-serv'
 'Transport-moving' 'Handlers-cleaners' 'Armed-Forces']
['Own-child' 'Husband' 'Not-

In [35]:
# анализ показал, что в ряде сторк есть неизвестные значения. Отбросим их
df1 = df[(df['workclass'] != '?') & (df['occupation'] != '?') & (df['native-country'] != '?')]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 16 columns):
age                45222 non-null int64
workclass          45222 non-null object
fnlwgt             45222 non-null int64
education          45222 non-null object
educational-num    45222 non-null int64
marital-status     45222 non-null object
occupation         45222 non-null object
relationship       45222 non-null object
race               45222 non-null object
gender             45222 non-null object
capital-gain       45222 non-null int64
capital-loss       45222 non-null int64
hours-per-week     45222 non-null int64
native-country     45222 non-null object
income             45222 non-null object
num_income         45222 non-null int64
dtypes: int64(7), object(9)
memory usage: 5.9+ MB


In [43]:
# Проверяем что не осталось неизвестных значений
for col in df1:
    print(col, df1[col].unique())

age [25 38 28 44 34 63 24 55 65 36 26 48 43 20 37 45 22 23 54 32 46 56 17 29
 39 52 18 21 42 33 30 47 41 19 69 50 31 59 49 58 40 27 57 61 51 73 53 80
 62 35 72 64 68 66 60 67 71 70 90 77 81 74 78 82 75 85 76 89 83 79 88 87
 84 86]
workclass ['Private' 'Local-gov' 'Self-emp-not-inc' 'Federal-gov' 'State-gov'
 'Self-emp-inc' 'Without-pay']
fnlwgt [226802  89814 336951 ... 199655 255835 257302]
education ['11th' 'HS-grad' 'Assoc-acdm' 'Some-college' '10th' 'Prof-school'
 '7th-8th' 'Bachelors' 'Masters' '5th-6th' 'Assoc-voc' '9th' 'Doctorate'
 '12th' '1st-4th' 'Preschool']
educational-num [ 7  9 12 10  6 15  4 13 14  3 11  5 16  8  2  1]
marital-status ['Never-married' 'Married-civ-spouse' 'Widowed' 'Separated' 'Divorced'
 'Married-spouse-absent' 'Married-AF-spouse']
occupation ['Machine-op-inspct' 'Farming-fishing' 'Protective-serv' 'Other-service'
 'Prof-specialty' 'Craft-repair' 'Adm-clerical' 'Exec-managerial'
 'Tech-support' 'Sales' 'Priv-house-serv' 'Transport-moving'
 'Handlers-clea

In [46]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [47]:
# Переводим искомые значению в числовую кодировку
le.fit(df1['income'])

LabelEncoder()

In [55]:
# Переводим искомые значению в числовую кодировку. Заносим в массив значения income
y = le.transform(df1.income)
y.shape

(45222,)

In [58]:
# выбираем колонки, которые будут служить параметрами модели
SelectedColumns = df1[['age', 'workclass', 'educational-num', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']]
SelectedColumns

,age,workclass,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
5,34,Private,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States
7,63,Self-emp-not-inc,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States
8,24,Private,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States
9,55,Private,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States
10,65,Private,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,6418,0,40,United-States
11,36,Federal-gov,13,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States


In [64]:
# применяем one hot encoding к текстовым значениям, проверяем состав параметров.
X = pd.get_dummies( SelectedColumns, columns = [ 'workclass','marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country'] )
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 87 columns):
age                                          45222 non-null int64
educational-num                              45222 non-null int64
capital-gain                                 45222 non-null int64
capital-loss                                 45222 non-null int64
hours-per-week                               45222 non-null int64
workclass_Federal-gov                        45222 non-null uint8
workclass_Local-gov                          45222 non-null uint8
workclass_Private                            45222 non-null uint8
workclass_Self-emp-inc                       45222 non-null uint8
workclass_Self-emp-not-inc                   45222 non-null uint8
workclass_State-gov                          45222 non-null uint8
workclass_Without-pay                        45222 non-null uint8
marital-status_Divorced                      45222 non-null uint8
marital-status_Married-AF-spouse 

In [86]:
from sklearn.model_selection import train_test_split

In [87]:
# разбиваем выборку на тренировочную и тестовые группы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [65]:
# вызываем модель логистической регрессии
model = LogisticRegression()

In [88]:
# обучаем модель 
model.fit(X_train,y_train)

D:\DS\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [105]:
from sklearn.model_selection import train_test_split

In [90]:
# проверяем точность модели на тренировочной выборке
model.score(X_train, y_train)

0.8462486179118622

In [92]:
# проверяем точность модели на тестовой выборке
model.score(X_test, y_test)

0.851256725878971

In [97]:
# строим модель SVM
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='auto',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [98]:
clf.score(X_train, y_train)

0.8532617280050545

In [100]:
clf.score(X_test, y_test)

0.8484558118965136

In [101]:
svc = SVC(gamma='auto')

In [102]:
# строим модель SVM без стандартизации параметров
svc.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [103]:
svc.score(X_train, y_train)

0.8676038540514927

In [104]:
svc.score(X_test, y_test)

0.8688730006633744